In [ ]:
%load_ext autoreload
%autoreload 2

# chat

> A simple interface for the Mistral API.

> This notebook follows the [fastai style guide](https://docs.fast.ai/dev/style.html). (perhaps move this to the index page only.)

In [ ]:
#| default_exp chat 

## API Key

Testing our API key to see check whether we have it working correctly.

In [ ]:
import os
API_KEY = os.environ.get('MISTRAL_API_KEY', 'KEY NOT FOUND')
message = 'Who is the most renowned French artist?'
h = {'Content-Type': 'application/json', 'Accept': 'application/json', 'Authorization': f'Bearer {API_KEY}'}
d = {'model': 'open-mistral-7b', 'messages': [{'role': 'user', 'content': message}]}

In [ ]:
import requests
URL = 'https://api.mistral.ai/v1/chat/completions'
r = requests.post(URL, headers=h, json=d); r.status_code

200

In [ ]:
# r.json()['choices'][0]['message']['content']

"There are many renowned French artists across various artistic mediums, but some of the most famous ones include:\n\n1. Leonardo da Vinci (1452-1519): Although he was an Italian artist, he spent his last few years in France, and some of his most famous works were completed there. The Mona Lisa and The Last Supper are among his most famous works.\n\n2. Claude Monet (1840-1926): Monet is considered the founder of Impressionism, a revolutionary art movement. Some of his most famous works include Water Lilies, Impression, Sunrise, and Rouen Cathedral series.\n\n3. Pablo Picasso (1881-1973): Picasso was a Spanish artist who lived and worked in France for most of his life. He is one of the most influential artists of the 20th century, and he co-founded the Cubist movement. Some of his most famous works include Les Demoiselles d'Avignon, Guernica, and The Old Guitarist.\n\n4. Henri Matisse (1869-1954): Matisse was a leading figure in Modernist painting and sculpture. He is known for his use 

Exploring how message history would work.

In [ ]:
m = 'How about from Britian?'
d['messages'][0]['content'] = m
r = requests.post(URL, headers=h, json=d)
# r.json()['choices'][0]['message']['content']

"I'd be happy to help answer any questions you have about Britain or the United Kingdom. Here are some common topics related to Britain that people often ask about:\n\n1. Geography: Where is Britain located? What countries make up the United Kingdom? What are the major cities in the UK?\n2. History: When was Britain founded? What significant historical events have taken place in Britain? Who were some famous British monarchs?\n3. Culture: What is British culture like? What are some traditional British foods? What are some popular British customs and traditions?\n4. Politics: What is the British government like? Who are the current political leaders in the UK? What are the major political parties in the UK?\n5. Education: What is the education system like in Britain? What are some famous British universities?\n6. Travel: What are some popular tourist destinations in Britain? What are some tips for traveling in the UK?\n7. Sports: What are some popular sports in Britain? Who are some fam

It seems that prior interactions are not stored. Therefore, we will have to do this ourselves.

## Mistral Library

The MistralAI library allows three different ways to use the API.
- No streaming.
- With streaming.
- Async.

In [ ]:
#| export
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [ ]:
client = MistralClient(api_key=API_KEY)
messages = [ChatMessage(role='user', content='What is the best French cheese?')]

### No Streaming

In [ ]:
m = 'open-mistral-7b'
chat_response = client.chat(model=m, messages=messages); chat_response

ChatCompletionResponse(id='d7c35eda5e7345ca9daf28cb956a7d08', object='chat.completion', created=1712839000, model='open-mistral-7b', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='Determining the "best" French cheese is subjective and depends on personal preferences. Here are a few renowned French cheeses that are widely appreciated for their unique flavors and textures:\n\n1. Roquefort: A blue-veined sheep\'s milk cheese from the Massif Central region. It has a strong, pungent flavor and a creamy texture.\n\n2. Comté: A nutty and slightly sweet cow\'s milk cheese from the Jura region. It has a firm texture and can be aged for various lengths of time.\n\n3. Camembert de Normandie: A soft, earthy cow\'s milk cheese from Normandy. It has a white bloom rind and a creamy, runny texture.\n\n4. Brie de Meaux: A soft, creamy cow\'s milk cheese from the Île-de-France region. It has a white, downy rind and a mild, buttery flavor.\n\n5. Munster: A p

In [ ]:
chat_response.choices[0].message.content

'Determining the "best" French cheese is subjective and depends on personal preferences. Here are a few renowned French cheeses that are widely appreciated for their unique flavors and textures:\n\n1. Roquefort: A blue-veined sheep\'s milk cheese from the Massif Central region. It has a strong, pungent flavor and a creamy texture.\n\n2. Comté: A nutty and slightly sweet cow\'s milk cheese from the Jura region. It has a firm texture and can be aged for various lengths of time.\n\n3. Camembert de Normandie: A soft, earthy cow\'s milk cheese from Normandy. It has a white bloom rind and a creamy, runny texture.\n\n4. Brie de Meaux: A soft, creamy cow\'s milk cheese from the Île-de-France region. It has a white, downy rind and a mild, buttery flavor.\n\n5. Munster: A pungent, smelly cow\'s milk cheese from the Alsace region. It has a gooey, runny texture and a strong, sulfurous aroma.\n\n6. Époisses de Bourgogne: A soft, pungent cow\'s milk cheese from Burgundy. It has a pink rind and a str

No streaming returns the entire message at once.

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()